In [3]:
from pathlib import Path
from kedro.framework.context import load_context

current_dir = Path.cwd()  # this points to 'notebooks/' folder
proj_path = current_dir.parent  # point back to the root of the project
context = load_context(proj_path)

In [4]:
context

In [12]:
df_train = catalog.load("house_price_training")

2020-07-26 23:04:49,646 - kedro.io.data_catalog - INFO - Loading data from `house_price_training` (CSVDataSet)...


In [17]:
df_test = catalog.load("house_price_test")

2020-07-26 23:06:11,567 - kedro.io.data_catalog - INFO - Loading data from `house_price_test` (CSVDataSet)...


In [18]:
training_data = df_train
input_columns = ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
                 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
                 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
                 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
                 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
                 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
                 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
                 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
                 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
                 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
                 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
                 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
                 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
                 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
                 'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
                 'SaleCondition']

target = "SalePrice"
train_y = training_data.loc[:, target]
train_x = training_data.loc[:, input_columns]
test_x = df_test.loc[:, input_columns]

In [19]:
test_x

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [20]:
from category_encoders import BinaryEncoder
categorical_columns_list = list(train_x.columns[train_x.dtypes==object])
ce_be = BinaryEncoder(cols=categorical_columns_list, handle_unknown="inpute")
training_data_ce_binary = ce_be.fit_transform(train_x)
test_data_ce_binary = ce_be.transform(test_x)

In [21]:
training_data_ce_binary.columns

Index(['MSSubClass', 'MSZoning_0', 'MSZoning_1', 'MSZoning_2', 'MSZoning_3',
       'LotFrontage', 'LotArea', 'Street_0', 'Street_1', 'Alley_0',
       ...
       'YrSold', 'SaleType_0', 'SaleType_1', 'SaleType_2', 'SaleType_3',
       'SaleType_4', 'SaleCondition_0', 'SaleCondition_1', 'SaleCondition_2',
       'SaleCondition_3'],
      dtype='object', length=199)

In [22]:
test_data_ce_binary.columns

Index(['MSSubClass', 'MSZoning_0', 'MSZoning_1', 'MSZoning_2', 'MSZoning_3',
       'LotFrontage', 'LotArea', 'Street_0', 'Street_1', 'Alley_0',
       ...
       'YrSold', 'SaleType_0', 'SaleType_1', 'SaleType_2', 'SaleType_3',
       'SaleType_4', 'SaleCondition_0', 'SaleCondition_1', 'SaleCondition_2',
       'SaleCondition_3'],
      dtype='object', length=199)

In [29]:
# import lightgbm as lgb
# reg = lgb.LGBMRegressor(n_estimators=10000)
# reg.fit(training_data_ce_binary, train_y)

LGBMRegressor(n_estimators=10000)

In [1]:
import featuretools as ft

ModuleNotFoundError: No module named 'featuretools'

In [2]:
import featuretools as ft

ModuleNotFoundError: No module named 'featuretools'